In [1]:
# 회귀모형 
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd 
import seaborn as sns 

# 쓸데없는 알림 방지
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

import itertools

#통계적 모형
import statsmodels.api as sm # 선형회귀 
from statsmodels.stats.outliers_influence import variance_inflation_factor # 다중공산성 

#머신러닝
from sklearn import datasets
from sklearn import metrics
from sklearn import svm, neighbors, tree  # 트리 KNN 

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier # 앙상블  
from sklearn.linear_model import Ridge, Lasso, LinearRegression, LogisticRegression #릿지라쏘 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold # 그리드 서치, 트레인 테스트 셋 분리 
from sklearn.preprocessing import StandardScaler, RobustScaler, Normalizer,MinMaxScaler # scale

from sklearn.metrics import mean_squared_log_error, mean_squared_error, r2_score, mean_absolute_error #연속형일때 사용하는 경우 
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, confusion_matrix #범주형(분류모델)

# boost 계열 
import xgboost as xgb 
import lightgbm as lgb
import catboost as cb

In [2]:
def acc(cmat):
    acc=(cmat[0,0]+cmat[1,1])/np.sum(cmat)
    return acc 

def checkSubset(x,y,feature_set):
    #model = sm.OLS(y, x[list(feature_set)]).fit()
    model = LogisticRegression(penalty='none').fit(x[list(feature_set)],y)
    r2 = model.score(x[list(feature_set),y])
    return {'model' : model, 'r2' : r2, 'columns': feature_set}


def forward(x,y):
    
    selected_columns = []

    for i in range(0, x.shape[1]):
        results = []
        forward_columns = [col for col in x.columns if col not in selected_columns]
        
        for col in forward_columns:
            ret = checkSubset(x=x, y=y, feature_set=selected_columns + [col])
            results.append(ret)

        models = pd.DataFrame(results)
        best_model = models.loc[models.r2.argmax()]
        
        if i > 0:
            if best_model.r2 < before_model.r2: break
            else: before_model = best_model
                
        else:
            # before_aic = best_model.model.aic
            before_model = best_model

        selected_columns = list(best_model.columns)
        # print('selected columns:', selected_columns, 'AIC:', best_model.model.aic)
        
    return before_model




def backward(x, y):
    
    selected_columns = list(x.columns)
    
    for i in range(0, x.shape[1]):
        results = []
        for combo in itertools.combinations(selected_columns, len(selected_columns)-1):
            backword_columns = list(combo)
            ret = checkSubset(x=x, y=y, feature_set=backword_columns)
            results.append(ret)

        models = pd.DataFrame(results)
        best_model = models.loc[models['r2'].argmax()]
        
        if i > 0:
            if best_model.r2 < before_model.r2: break
            else: before_model = best_model
        else:
            # before_aic = best_model.model.aic
            before_model = best_model
            
        selected_columns = list(best_model.columns)
    
    return before_model


In [3]:
def forward_select(x, y, selected_columns):
    
    results = []
    forward_columns = [a for a in x.columns if a not in selected_columns]
    
    for col in forward_columns:
        ret = checkSubset(x=x, y=y, feature_set=selected_columns + [col])
        results.append(ret)

    models = pd.DataFrame(results)
    best_model = models.loc[models.r2.argmax()]
    
    return best_model


def backward_drop(x, y, selected_columns):
    results = []
    for combo in itertools.combinations(selected_columns, len(selected_columns)-1):
        backword_columns = list(combo)
        ret = checkSubset(x=x, y=y, feature_set=backword_columns)
        results.append(ret)

    models = pd.DataFrame(results)
    best_model = models.loc[models['r2'].argmax()]
    
    return best_model

def stepwise(x, y):
    selected_columns = []
        
    for i in range(0, x.shape[1]):
        forward_model = forward_select(x=x, y=y, selected_columns=selected_columns)
        selected_columns = list(forward_model.columns)
        
        if i < 2:  before_model = forward_model; continue

        large_model = forward_model
        backward_model = backward_drop(x=x, y=y, selected_columns=selected_columns)
        
        if forward_model.r2 < backward_model.r2:
            selected_columns = list(backward_model.columns)
            large_model = backward_model

        if before_model.r2 < large_model.r2:
            before_model = large_model
        else: 
            break
    
    return before_model


In [4]:
model = LogisticRegression().fit(titanic_features, titanic_raw_data.Survived)
model = forward(x=titanic_features, y=titanic_target)
model

NameError: name 'titanic_features' is not defined

In [5]:
titanic_raw_data = pd.read_csv('C:/Users/scien/Videos/titanic/titanic_full_data.csv')

In [6]:
# 반응변수 Y를 만들자 
titanic_target = titanic_raw_data[['Survived']]

In [7]:
titanic_target

,Survived
0,0
1,1
2,1
3,1
4,0
...,...
1280,0
1281,1
1282,0
1283,0


In [8]:
# Y 해당하는 survived 제외, 문자열 타입 Name, Cabin 제외 
columns = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch','Fare', 'Embarked']

In [9]:
# 입력변수 X를 만들자 
titanic_features = titanic_raw_data[columns]
titanic_features

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...
1280,3,male,NaN,0,0,8.0500,S
1281,1,female,39.0,0,0,108.9000,C
1282,3,male,38.5,0,0,7.2500,S
1283,3,male,NaN,0,0,8.0500,S


In [10]:
# 문자열 자료를 이산형으로 만들기 
# 굳이 2개 다 쓸 필요가 없어 하나만 이용할 것임 drop_first = True
# 만일 female을 남기고 싶다면?
# columns=['male', 'female']

titanic_features[['feamle', 'male']] = pd.get_dummies(titanic_features['Sex'])

In [11]:
titanic_features.drop(columns=['Sex'], inplace=True)

In [12]:
titanic_features

,Pclass,Age,SibSp,Parch,Fare,Embarked,feamle,male
0,3,22.0,1,0,7.2500,S,0,1
1,1,38.0,1,0,71.2833,C,1,0
2,3,26.0,0,0,7.9250,S,1,0
3,1,35.0,1,0,53.1000,S,1,0
4,3,35.0,0,0,8.0500,S,0,1
...,...,...,...,...,...,...,...,...
1280,3,NaN,0,0,8.0500,S,0,1
1281,1,39.0,0,0,108.9000,C,1,0
1282,3,38.5,0,0,7.2500,S,0,1
1283,3,NaN,0,0,8.0500,S,0,1


In [13]:
titanic_features['Embarked'].unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [14]:
dummy_columns = pd.get_dummies(titanic_features['Embarked'], prefix='Embarked').columns

In [15]:
titanic_features[dummy_columns] = pd.get_dummies(titanic_features['Embarked'], prefix='Embarked')

In [16]:
titanic_features.drop(columns=['Embarked'], inplace=True)

In [17]:
# pd.cut(data, bins, label) n개의 구간별로 나누기 pd.cut

bins = [0, 10, 20, 30, 40, 50, 60, 70, 80] # 시작과 끝값을 지정
labels = ['child', 'teens', '20s', '30s', '40s', '50s', '60s', '70s']

pd.cut(titanic_features['Age'], bins=bins, labels=labels).value_counts()

# 결측치 삭제 필요 

20s      357
30s      209
teens    156
40s      128
child     85
50s       62
60s       27
70s        6
Name: Age, dtype: int64

In [18]:
# 나이의 결측치 
titanic_features.Age.isna().sum()

255

In [19]:
# 평균 30살 
round(titanic_features.Age.mean())

30

In [20]:


# 나이 결측치를 평균 나이로 채워주기 
titanic_features.Age.fillna(round(titanic_features.Age.mean()), inplace=True)

In [21]:
titanic_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1285 entries, 0 to 1284
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      1285 non-null   int64  
 1   Age         1285 non-null   float64
 2   SibSp       1285 non-null   int64  
 3   Parch       1285 non-null   int64  
 4   Fare        1284 non-null   float64
 5   feamle      1285 non-null   uint8  
 6   male        1285 non-null   uint8  
 7   Embarked_C  1285 non-null   uint8  
 8   Embarked_Q  1285 non-null   uint8  
 9   Embarked_S  1285 non-null   uint8  
dtypes: float64(2), int64(3), uint8(5)
memory usage: 56.6 KB


In [22]:
# Fare의 결측치 

titanic_features.loc[titanic_features.Fare.isna()]

,Pclass,Age,SibSp,Parch,Fare,feamle,male,Embarked_C,Embarked_Q,Embarked_S
1035,3,60.5,0,0,NaN,0,1,0,0,1


In [23]:
# 3등실의 평균 Fare는?
titanic_features.loc[titanic_features.Pclass == 3]['Fare'].mean()

13.269605772005766

In [24]:
titanic_features.Fare.fillna(round(titanic_features.loc[titanic_features.Pclass == 3]['Fare'].mean(), 4), inplace=True)

In [25]:
# 모든 결측치를 채웠다 
titanic_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1285 entries, 0 to 1284
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      1285 non-null   int64  
 1   Age         1285 non-null   float64
 2   SibSp       1285 non-null   int64  
 3   Parch       1285 non-null   int64  
 4   Fare        1285 non-null   float64
 5   feamle      1285 non-null   uint8  
 6   male        1285 non-null   uint8  
 7   Embarked_C  1285 non-null   uint8  
 8   Embarked_Q  1285 non-null   uint8  
 9   Embarked_S  1285 non-null   uint8  
dtypes: float64(2), int64(3), uint8(5)
memory usage: 56.6 KB


In [26]:
print(titanic_features.isna().sum())
print(titanic_target.isna().sum())

Pclass        0
Age           0
SibSp         0
Parch         0
Fare          0
feamle        0
male          0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
dtype: int64
Survived    0
dtype: int64


In [27]:
titanic_features.corr()

,Pclass,Age,SibSp,Parch,Fare,feamle,male,Embarked_C,Embarked_Q,Embarked_S
Pclass,1.000000,-0.365455,0.061054,0.013556,-0.559074,-0.137154,0.137154,-0.278352,0.229773,0.104158
Age,-0.365455,1.000000,-0.189013,-0.131399,0.169417,-0.055277,0.055277,0.078611,-0.012696,-0.066401
SibSp,0.061054,-0.189013,1.000000,0.376551,0.160461,0.109707,-0.109707,-0.048486,-0.046295,0.073729
Parch,0.013556,-0.131399,0.376551,1.000000,0.225806,0.206756,-0.206756,-0.007369,-0.097484,0.069945
Fare,-0.559074,0.169417,0.160461,0.225806,1.000000,0.192373,-0.192373,0.292137,-0.129290,-0.177973
feamle,-0.137154,-0.055277,0.109707,0.206756,0.192373,1.000000,-1.000000,0.065795,0.083968,-0.115917
male,0.137154,0.055277,-0.109707,-0.206756,-0.192373,-1.000000,1.000000,-0.065795,-0.083968,0.115917
Embarked_C,-0.278352,0.078611,-0.048486,-0.007369,0.292137,0.065795,-0.065795,1.000000,-0.162420,-0.775193
Embarked_Q,0.229773,-0.012696,-0.046295,-0.097484,-0.129290,0.083968,-0.083968,-0.162420,1.000000,-0.491614
Embarked_S,0.104158,-0.066401,0.073729,0.069945,-0.177973,-0.115917,0.115917,-0.775193,-0.491614,1.000000


In [28]:
#sns.pairplot(titanic_features)

### vif를 통한 다중공산성 확인 

In [29]:
vif = pd.DataFrame()
vif['vif factor'] = [variance_inflation_factor(titanic_features.values, i) for i in range(titanic_features.shape[1])]
vif['columns'] = titanic_features.columns
vif_original = vif 
vif_original

,vif factor,columns
0,1.801265,Pclass
1,1.218105,Age
2,1.225885,SibSp
3,1.277321,Parch
4,1.686548,Fare
5,236.134896,feamle
6,442.665646,male
7,105.771999,Embarked_C
8,55.911813,Embarked_Q
9,136.259755,Embarked_S


## 다중공산성 기준으로 뽑은 변수선택법

In [30]:
data_vif_feature = titanic_features[['Pclass', 'Age', 'SibSp','Parch', 'Fare', 'feamle', 'Embarked_C','Embarked_Q']]
vif = pd.DataFrame()
vif['vif factor'] = [variance_inflation_factor(data_vif_feature.values, i) for i in range(data_vif_feature.shape[1])]
vif['feature'] = data_vif_feature.columns
vif

,vif factor,feature
0,3.955232,Pclass
1,4.160751,Age
2,1.500960,SibSp
3,1.520710,Parch
4,1.999517,Fare
5,1.626260,feamle
6,1.399030,Embarked_C
7,1.211157,Embarked_Q


## AIC를 기준으로 한 변수선택
- 파이썬에서 알아보기 어렵다.
- 따라서 그 뭐냐 R에서 알아봤다 131501 파일 참조할 것 

In [32]:
data_f_aic_features = titanic_features[['feamle' ,'Pclass' , 'Age' , 'SibSp' , 'Embarked_C']]
data_b_aic_features = titanic_features[['Pclass', 'Age', 'SibSp', 'feamle', 'Embarked_Q', 'Embarked_S']]

## R-square를 기준으로 한 변수선택 

In [34]:
f_model = forward(x=titanic_features, y=titanic_target)

TypeError: '(['Pclass'],       Survived
0            0
1            1
2            1
3            1
4            0
...        ...
1280         0
1281         1
1282         0
1283         0
1284         1

[1285 rows x 1 columns])' is an invalid key

In [200]:
b_model = backward(x=titanic_features, y=titanic_target)

TypeError: '(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'feamle', 'male', 'Embarked_C', 'Embarked_Q'],       Survived
0            0
1            1
2            1
3            1
4            0
...        ...
1280         0
1281         1
1282         0
1283         0
1284         1

[1285 rows x 1 columns])' is an invalid key

In [190]:
step_model = stepwise(x=titanic_features, y=titanic_target)

TypeError: '(['Pclass'],       Survived
0            0
1            1
2            1
3            1
4            0
...        ...
1280         0
1281         1
1282         0
1283         0
1284         1

[1285 rows x 1 columns])' is an invalid key

In [45]:
#data_forward_r2_features = titanic_features[f_model.columns]
#data_backward_r2_features = titanic_features[b_model.columns]
#data_both_r2_feature = titanic_features[step_model.columns]

In [41]:
data_vif_feature
#data_forward_aic_feature
#data_forward_r2_feature
data_both_r2_feature
data_backward_doc_feature
data_backward_doc_feature

NameError: name 'data_forward_doc_feature' is not defined

#### r 스퀘어 확인 

In [104]:
model.score(titanic_features, titanic_target)

0.7898832684824902

# 여기서 중요! 

- 위에서 살펴본 정리된 데이터를 이곳에서 학습셋, 연습셋으로 스플릿 할 것이다. 
- VIF로 따져본 걸 X,Y에 투입하거나 
- 전진, 후진, 단계적 선택법 AIC를 투입?
- 전 후 단계적 r2를 투입하건 ㅏ

In [119]:
train_x, test_x, train_y, test_y = train_test_split(data_f_aic_features, titanic_target, train_size=0.8, random_state = 123)
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

(1028, 5) (257, 5) (1028, 1) (257, 1)


### 모델을 적합하기 전에 스케일링이 필요하다. 
- 스탠다드 스케일러 

In [120]:
std_scaler = StandardScaler()
std_scaler_fit = std_scaler.fit(train_x)
train_x_scale = pd.DataFrame(std_scaler_fit.transform(train_x), index=train_x.index, columns=train_x.columns)
# x의 값을 스케일링 해주고, X와 Y좌표를 X의 데이터 그대로 적합시켜준다.


test_x_scale = pd.DataFrame(std_scaler_fit.transform(test_x), index = test_x.index, columns=test_x.columns)

### min max sclaer (가장 많이 사용하는 스케일링)

In [106]:
std_scaler = MinMaxScaler()
std_scaler_fit = std_scaler.fit(train_x)
train_x_scale = pd.DataFrame(std_scaler_fit.transform(train_x), index=train_x.index, columns=train_x.columns)
test_x_scale = pd.DataFrame(std_scaler_fit.transform(test_x), index = test_x.index, columns=test_x.columns)

### RobustCaler 

In [110]:
std_scaler = RobustScaler()
std_scaler_fit = std_scaler.fit(train_x)
train_x_scale = pd.DataFrame(std_scaler_fit.transform(train_x), index=train_x.index, columns=train_x.columns)
test_x_scale = pd.DataFrame(std_scaler_fit.transform(test_x), index = test_x.index, columns=test_x.columns)

### R에서 사용하던 노멀라이저 스케일링 

In [114]:
std_scaler = Normalizer()
std_scaler_fit = std_scaler.fit(train_x)
train_x_scale = pd.DataFrame(std_scaler_fit.transform(train_x), index=train_x.index, columns=train_x.columns)
test_x_scale = pd.DataFrame(std_scaler_fit.transform(test_x), index = test_x.index, columns=test_x.columns)

## SVM 
- C값은?
- 커널은? 
- 감마는?
- 반복(max_itre)은?
- gamma값은 rbf에만 있다.

#### 학습 데이터를 가지고 svm에 필요한 요소값을 찾아보자 

In [99]:
param = {
    'C' : [0.001, 0.01, 0.1, 1, 10,100],
    'gamma' : [0.001,0.01,0.1,1,10,100]
}

svc = svm.SVC(kernel='rbf')
ret = GridSearchCV(svc, param_grid=param, cv=5, return_train_score=True).fit(train_x, train_y)

In [100]:
print(ret.best_params_)
print(ret.best_score_)

{'C': 100, 'gamma': 0.001}
0.7675065119583234


In [101]:
model_vif_normal = ret.best_estimator_

#### 스탠다드 스케일한 데이터를 가지고 svm에 필요한 요소값을 찾아보자 

In [103]:
param = {
    'C' : [0.001, 0.01, 0.1, 1, 10,100],
    'gamma' : [0.001,0.01,0.1,1,10,100]
}

svc = svm.SVC(kernel='rbf')
ret = GridSearchCV(svc, param_grid=param, cv=5, return_train_score=True).fit(train_x_scale, train_y)

In [104]:
print(ret.best_params_)
print(ret.best_score_)

{'C': 1, 'gamma': 0.1}
0.7928202699502723


In [105]:
model_vif_std = ret.best_estimator_

#### min max 스케일한 데이터를 가지고 svm에 필요한 요소값을 찾아보자 

In [107]:
param = {
    'C' : [0.001, 0.01, 0.1, 1, 10,100],
    'gamma' : [0.001,0.01,0.1,1,10,100]
}

svc = svm.SVC(kernel='rbf')
ret = GridSearchCV(svc, param_grid=param, cv=5, return_train_score=True).fit(train_x_scale, train_y)

In [108]:
print(ret.best_params_)
print(ret.best_score_)

{'C': 1, 'gamma': 1}
0.8006014681506037


In [109]:
model_vif_mm = ret.best_estimator_

#### robust 스케일한 데이터를 가지고 svm에 필요한 요소값을 찾아보자 

In [111]:
param = {
    'C' : [0.001, 0.01, 0.1, 1, 10,100],
    'gamma' : [0.001,0.01,0.1,1,10,100]
}

svc = svm.SVC(kernel='rbf')
ret = GridSearchCV(svc, param_grid=param, cv=5, return_train_score=True).fit(train_x_scale, train_y)

In [112]:
print(ret.best_params_)
print(ret.best_score_)

{'C': 100, 'gamma': 0.01}
0.7908785223774567


In [113]:
model_vif_rb = ret.best_estimator_

#### R 정규화로 스케일한 데이터를 가지고 svm에 필요한 요소값을 찾아보자 

In [115]:
param = {
    'C' : [0.001, 0.01, 0.1, 1, 10,100],
    'gamma' : [0.001,0.01,0.1,1,10,100]
}

svc = svm.SVC(kernel='rbf')
ret = GridSearchCV(svc, param_grid=param, cv=5, return_train_score=True).fit(train_x_scale, train_y)

In [116]:
print(ret.best_params_)
print(ret.best_score_)

{'C': 100, 'gamma': 10}
0.772365616860052


In [117]:
model_vif_nm = ret.best_estimator_

#### 계단 ,AIC로 파악하기 

In [121]:
param = {
    'C' : [0.001, 0.01, 0.1, 1, 10,100],
    'gamma' : [0.001,0.01,0.1,1,10,100]
}

svc = svm.SVC(kernel='rbf')
ret = GridSearchCV(svc, param_grid=param, cv=5, return_train_score=True).fit(train_x_scale, train_y)

In [122]:
print(ret.best_params_)
print(ret.best_score_)

{'C': 10, 'gamma': 0.1}
0.7928013260715131


In [123]:
ret.best_estimator_

SVC(C=10, gamma=0.1)